Importing libraries for georeferencing purposes

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0   conda-forge
    geopy:         1.18.1-py_0 conda-forge

geographiclib- 100% |################################| Time: 0:00:00  23.59 MB/s
geopy-1.18.1-p 100% |################################| Time: 0:00:00  37.08 MB/s
Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    altair:  2.2.2-py35_1 conda-forge
    branca:  0.3.1-py_0   conda-forge
    folium:  0.5.0-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forge

altair-2.2.2-p 100% |################################| Time: 0:00:00  56.79 MB/s
branca-0.3.1-p 100% |################################| Time: 0:00:00  28.61 MB/s
vincent-0.4.4- 100% |###################

Authorizing with "Foursquare"

In [17]:
CLIENT_ID = 'DZOZJVJU35C43HQTP2DK2WFTFHH4LOB3OGR2A4FR1VXDA0UM' # your Foursquare ID
CLIENT_SECRET = 'JORWUPXN5RSQRYSTHZYE2W0233DLGYPLLWAZCFRX0CNIWXBD' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: DZOZJVJU35C43HQTP2DK2WFTFHH4LOB3OGR2A4FR1VXDA0UM
CLIENT_SECRET:JORWUPXN5RSQRYSTHZYE2W0233DLGYPLLWAZCFRX0CNIWXBD


Import the file with Postal Codes for Calgary, AL

In [140]:

body = client_2601683365d34a23a3ccc6f999e0ef77.get_object(Bucket='capstoneproject-donotdelete-pr-b8xqkx5kjieiq4',Key='code1.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_16 = pd.read_csv(body)
df_data_16.head()



,Place,Code,City
0,City Centre / Tower,T2P,Calgary
1,Cranston,T3M,Calgary
2,Northeast,T3N,Calgary
3,Northwest,T3R,Calgary
4,Rundle / Whitehorn / Monterey Park,T1Y,Calgary


Importing the file with Calgary public schools

In [141]:
body = client_2601683365d34a23a3ccc6f999e0ef77.get_object(Bucket='capstoneproject-donotdelete-pr-b8xqkx5kjieiq4',Key='school_loc4.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_18 = pd.read_csv(body)
df_data_18.head()

,P_Code,NAME,BOARD,TYPE,NAME_AB,ADDRESS_AB,CITY,POSTAL_COD,longitude,latitude,STRUCTURE_ID
0,T2T,Alternative High,Calgary School District No. 19,Public School,Alternative High School,5003 20 Street SW,Calgary,T2T5A5,-114.109960,51.009659,1382202
1,T1Y,Clarence Sansom,Calgary School District No. 19,Public School,Clarence Sansom School,5840 24 Avenue NE,Calgary,T1Y6G4,-113.949196,51.074613,1383016
2,T2X,Trinity Christian,Palliser Regional Division No. 26,Public School,Trinity Christian School,#100 295 Midpark Way S.E.,Calgary,T2X2A8,-114.062240,50.909328,1551635
3,T3E,St. Andrew,Calgary Roman Catholic Separate School Distric...,Separate School,St. Andrew School,4331 41 Ave SW,Calgary,T3E1G2,-114.147724,51.016363,1382283
4,T3C,Wildwood,Calgary School District No. 19,Public School,Wildwood School,120 45 Street SW,Calgary,T3C2B3,-114.152117,51.051590,1382147


Joining both tables in order to get a full observation of schools in the particular communities

In [142]:
result = pd.merge(df_data_18,
                 df_data_16,
                 left_on = 'P_Code',
                 right_on = 'Code',
                 how = 'left')

merge_table = result[['NAME','BOARD', 'TYPE', 'ADDRESS_AB','POSTAL_COD','Place','latitude', 'longitude']]
merge_table

,NAME,BOARD,TYPE,ADDRESS_AB,POSTAL_COD,Place,latitude,longitude
0,Alternative High,Calgary School District No. 19,Public School,5003 20 Street SW,T2T5A5,South Altadore / Bankview / Richmond,51.009659,-114.109960
1,Clarence Sansom,Calgary School District No. 19,Public School,5840 24 Avenue NE,T1Y6G4,Rundle / Whitehorn / Monterey Park,51.074613,-113.949196
2,Trinity Christian,Palliser Regional Division No. 26,Public School,#100 295 Midpark Way S.E.,T2X2A8,Midnapore / Sundance,50.909328,-114.062240
3,St. Andrew,Calgary Roman Catholic Separate School Distric...,Separate School,4331 41 Ave SW,T3E1G2,Lakeview / Glendale / Killarney / Glamorgan,51.016363,-114.147724
4,Wildwood,Calgary School District No. 19,Public School,120 45 Street SW,T3C2B3,Rosscarrock / Wildwood / Shaganappi / Sunalta,51.051590,-114.152117
5,Earl Grey,Calgary School District No. 19,Public School,845 Hillcrest Avenue SW,T2T0Z1,South Altadore / Bankview / Richmond,51.030874,-114.082654
6,Little Angels,Asasa Learning,Private School,599 Northmount Dr NW,T2K3J6,Thornecliffe / Tuxedo,51.087173,-114.088022
7,Maria Montessori,Maria Montessori Education Centre of Calgary Ltd.,Private School,Chief Crowfoot School 2634 12 Ave NW,T2N1K6,Kensington / Westmont / Parkdale / University,51.062710,-114.123583
8,Calgary Christian High,Palliser Regional Division No. 26,Public School,5029 - 26 Avenue S.W.,T3E0R5,Lakeview / Glendale / Killarney / Glamorgan,51.029763,-114.158941
9,Balmoral,Calgary School District No. 19,Public School,220 16 Avenue NW,T2M0H4,Mount Pleasant / Capitol Hill / Banff Trail,51.067882,-114.066480


Obtaining the georeferencing info for the City of Calgary with Nominatim method

In [30]:
address = 'Calgary, AL'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Calgary are {}, {}.'.format(latitude, longitude))


/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  app.launch_new_instance()


The geograpical coordinate of Calgary are 51.02532675, -114.049868485806.


Getting the total number of wards and communitied within the City of Calgary

In [33]:
print('The dataframe has {} wards and {} communities.'.format(
        len(result['Place'].unique()),
        result.shape[0]
    )
)


The dataframe has 35 wards and 460 communities.


Visualizing the schools on the map

In [139]:
map_calgary = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, school, community in zip(pd.to_numeric(merge_table.latitude, errors='coerce'), pd.to_numeric(merge_table.longitude, errors='coerce'), result['BOARD'],result['Place']):
    label = '{}, {}'.format(school, community)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_calgary)  
    
map_calgary

Picking the particular area, which is number 380 in order to analyze details

In [134]:
sel_com = merge_table.loc[380, 'Place']
sel_com

'Sandstone / Harvest Hills / Coventry Hills / Panorama Hills / Beddington'

In [ ]:
Finding out what schools are there around

In [118]:
school_lat = merge_table.loc[0, 'latitude'] 
school_long = merge_table.loc[0, 'longitude'] 
school_name = merge_table.loc[0, 'BOARD'] 

print('Latitude and longitude values of {} are {}, {}.'.format(school_name, 
                                                               school_lat, 
                                                               school_long))



Latitude and longitude values of Calgary School District No. 19 are 51.0096594597176, -114.109959974644.


Observation the amenities in the radius of 700 meters from the selected school

In [119]:
LIMIT = 50 # limit of number of venues returned by Foursquare API
radius = 700 # define radius
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    school_lat, 
    school_long, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=DZOZJVJU35C43HQTP2DK2WFTFHH4LOB3OGR2A4FR1VXDA0UM&client_secret=JORWUPXN5RSQRYSTHZYE2W0233DLGYPLLWAZCFRX0CNIWXBD&v=20180604&ll=51.0096594597176,-114.109959974644&radius=700&limit=50'

Viewing the results

In [120]:
final_results = requests.get(url).json()
final_results

{'meta': {'code': 200, 'requestId': '5c17e0b5dd57970688a30028'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-4b0586eaf964a520f27422e3-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/arts_entertainment/museum_history_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d190941735',
         'name': 'History Museum',
         'pluralName': 'History Museums',
         'primary': True,
         'shortName': 'History Museum'}],
       'id': '4b0586eaf964a520f27422e3',
       'location': {'address': '4520 Crowchild Tr SW',
        'cc': 'CA',
        'city': 'Calgary',
        'country': 'Canada',
        'distance': 647,
        'formattedAddress': ['4520 Crowchild Tr SW',
         'Calgary AB T2T 5J4',
         'Canada'],
        'labeledLatLngs': [{'label': 'di

In [ ]:
use function that extracts the category of the venue

In [122]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

clean up the json and structure it into a pandas dataframe

In [124]:
venues = final_results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues

,name,categories,lat,lng
0,The Military Museums,History Museum,51.013866,-114.116351
1,Flames Community Arenas,Skating Rink,51.011817,-114.116466
2,Glenmore Aquatic Centre,Pool,51.006528,-114.105701
3,Glenmore Athletic Park,Park,51.007002,-114.104787
4,Stu Peppard Arena,Hockey Rink,51.006543,-114.103551
5,Pegasus Restaurant,Greek Restaurant,51.010767,-114.100119


Counting the venues

In [125]:
print('{} venues are shown by Foursquare.'.format(nearby_venues.shape[0]))

6 venues are shown by Foursquare.


Finding out the coordinates of communities

In [126]:
commun_lat = merge_table.loc[0, 'latitude'] 
commun_long = merge_table.loc[0, 'longitude'] 
commun_name = merge_table.loc[0, 'Place'] 

print('Latitude and longitude values of {} are {}, {}.'.format(commun_name, 
                                                               commun_lat, 
                                                               commun_long))




Latitude and longitude values of South Altadore / Bankview / Richmond are 51.0096594597176, -114.109959974644.


Observe the venues around 1000 meters around target communities

In [128]:
LIMIT = 50 # limit of number of venues returned by Foursquare API
radius = 1000 # define radius
url1 = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    commun_lat, 
    commun_long, 
    radius, 
    LIMIT)
url1

'https://api.foursquare.com/v2/venues/explore?&client_id=DZOZJVJU35C43HQTP2DK2WFTFHH4LOB3OGR2A4FR1VXDA0UM&client_secret=JORWUPXN5RSQRYSTHZYE2W0233DLGYPLLWAZCFRX0CNIWXBD&v=20180604&ll=51.0096594597176,-114.109959974644&radius=1000&limit=50'

Getting the results

In [129]:
final_results = requests.get(url1).json()
final_results

{'meta': {'code': 200, 'requestId': '5c17e35adb04f54e69b64089'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-5502fb8c498e95778d8e7fd5-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/coffeeshop_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d1e0931735',
         'name': 'Coffee Shop',
         'pluralName': 'Coffee Shops',
         'primary': True,
         'shortName': 'Coffee Shop'}],
       'id': '5502fb8c498e95778d8e7fd5',
       'location': {'address': '4814 16 Street SW',
        'cc': 'CA',
        'city': 'Calgary',
        'country': 'Canada',
        'distance': 708,
        'formattedAddress': ['4814 16 Street SW', 'Calgary AB', 'Canada'],
        'labeledLatLngs': [{'label': 'display',
          'lat': 51.010967065214984,
          'lng

use function that extracts the category of the venue

In [130]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

clean up the json and structure it into a pandas dataframe

In [131]:
venues = final_results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues

,name,categories,lat,lng
0,Monogram Coffee,Coffee Shop,51.010967,-114.100054
1,The Military Museums,History Museum,51.013866,-114.116351
2,Garrison Pub,Pub,51.016131,-114.108674
3,Pfanntastic Pannenkoek Haus,Breakfast Spot,51.004185,-114.117476
4,My Favorite Ice Cream Shoppe,Ice Cream Shop,51.016239,-114.108888
5,Flames Community Arenas,Skating Rink,51.011817,-114.116466
6,Glenmore Aquatic Centre,Pool,51.006528,-114.105701
7,Glenmore Athletic Park,Park,51.007002,-114.104787
8,Stu Peppard Arena,Hockey Rink,51.006543,-114.103551
9,Pegasus Restaurant,Greek Restaurant,51.010767,-114.100119


Exposing the table describing which school belongs to a particular community

In [138]:
downtown_onehot = pd.get_dummies(merge_table[['Place']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
downtown_onehot['BOARD'] = merge_table['BOARD'] 

# move neighborhood column to the first column
fixed_columns = [downtown_onehot.columns[-1]] + list(downtown_onehot.columns[:-1])
downtown_onehot = downtown_onehot[fixed_columns]

downtown_onehot.head()



,BOARD,Braeside / Woodbine,Brentwood / Collingwood / Nose Hill,Bridgeland / Greenview / Zoo / YYC,City Centre / Tower,Connaught / West Victoria Park,Cranston,Dalhousie / Edgemont / Hamptons / Hidden Valley,Discovery Ridge / Signal Hill / Aspen Woods / Patterson / Cougar Ridge,Douglas Glen / McKenzie Lake / Copperfield / East Shepard,Elbow Park / Britannia / Parkhill / Mission,Forest Lawn / Dover / Erin Woods,Hawkwood / Arbour Lake / Royal Oak / Rocky Ridge,Highfield / Burns Industrial,Inglewood / Burnsland / Chinatown / East Victoria Park / Saddledome,Kensington / Westmont / Parkdale / University,Lakeview / Glendale / Killarney / Glamorgan,Lynnwood Ridge / Ogden / Foothills Industrial / Great Plains,Martindale / Taradale / Falconridge / Saddle Ridge,Midnapore / Sundance,Millrise / Somerset / Bridlewood / Evergreen,Montgomery / Bowness / Silver Springs / Greenwood,Mount Pleasant / Capitol Hill / Banff Trail,Northeast,Northwest,Oak Ridge / Haysboro / Kingsland / Windsor Park,Penbrooke Meadows / Marlborough,Queensland Downs / Lake Bonavista / Willow Park / Acadia,Rosscarrock / Wildwood / Shaganappi / Sunalta,Rundle / Whitehorn / Monterey Park,Sandstone / Harvest Hills / Coventry Hills / Panorama Hills / Beddington,South Altadore / Bankview / Richmond,Symons Valley,Thornecliffe / Tuxedo,Tuscany / Scenic Acres
0,Calgary School District No. 19,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
1,Calgary School District No. 19,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
2,Palliser Regional Division No. 26,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Calgary Roman Catholic Separate School Distric...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Calgary School District No. 19,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
